## Pandas Demo

In [3]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

### 文件读写 I/O

In [4]:
df = pd.read_csv('./data/ad_feature.csv', sep=',', header=0)    # 读入数据

In [5]:
df.shape

(846811, 6)

In [6]:
df.columns

Index(['adgroup_id', 'cate_id', 'campaign_id', 'customer', 'brand', 'price'], dtype='object')

In [7]:
df.head(10)

,adgroup_id,cate_id,campaign_id,customer,brand,price
0,63133,6406,83237,1,95471.0,170.00
1,313401,6406,83237,1,87331.0,199.00
2,248909,392,83237,1,32233.0,38.00
3,208458,392,83237,1,174374.0,139.00
4,110847,7211,135256,2,145952.0,32.99
5,607788,6261,387991,6,207800.0,199.00
6,375706,4520,387991,6,NaN,99.00
7,11115,7213,139747,9,186847.0,33.00
8,24484,7207,139744,9,186847.0,19.00
9,28589,5953,395195,13,NaN,428.00


In [8]:
df[:100].to_csv('./data/output.csv', index=False)    # 输出成CSV格式，索引（index）不要输出

In [9]:
df[:100].to_html('./data/output.html', index=False)    # 输出成网页table

In [10]:
writer = pd.ExcelWriter('./data/output.xlsx')    # 输出到Excel文件
df[:100].to_excel(writer, 'Sheet1', index=False)
df[100:200].to_excel(writer, 'Sheet2', index=False)
writer.save()

AttributeError: 'OpenpyxlWriter' object has no attribute 'save'

### 基本操作 Essential Basic Functionality

In [ ]:
df.head(10)    # 前十行

In [ ]:
df.tail(10)    # 倒数十行

In [ ]:
df['price'].describe()    # 一列的统计信息，只对于可以比较大小的值有意义

In [ ]:
df['cate_id'].describe()    # 类别，无意义

In [ ]:
# 迭代 Iteration, 一般用不到, 用于精细调整数据
for row in df[:10].itertuples():
    print(row.cate_id, row.price)

In [ ]:
# 排序 Sort
df[:10].sort_values(by='price')    # 按照价格从高到低排序

In [ ]:
df[:10].sort_values(by=['price', 'brand'], ascending=False)     # 可以按照多个key进行排序

In [ ]:
df[(df['price'] > 1000) & (df['price'] <= 1005)]    # 按条件选取

### 数据分组 Groupby

In [ ]:
df = pd.read_csv('./data/ad_feature.csv', sep=',', header=0, nrows=1000)
cate_group = df.groupby(['cate_id', 'customer'])    # 按照cate_id进行分组，把cate_id相同的数据分成一组
type(cate_group)

In [ ]:
cate_group.size()

In [ ]:
cate_group['price', 'brand'].mean().head()   # 计算每一组price和brand的平均值，必须保证函数在每个组中产生一个值

In [ ]:
df.groupby('cate_id')['brand'].mean().reset_index(drop=False).head()    # reset_index会重新设置index，drop参数控制是否丢弃原来的index

### 数据合并（Merge, Concatenate）

In [ ]:
df = pd.read_csv('./data/ad_feature.csv', sep=',', header=0)
df1 = df[['adgroup_id', 'price']]
df2 = df[['adgroup_id', 'cate_id', 'brand']]

In [ ]:
df1.head()

In [ ]:
df2.head()

In [ ]:
df3 = pd.merge(df1, df2, on='adgroup_id', how='inner')     # 把adgroup_id相同的数据拼接
df3.head()

In [ ]:
pd.concat([df[:4], df[10:15]], axis=0)    # 按照axis轴进行拼接，axis=0在行上拼接，axis=1在列上拼接

In [ ]:
pd.concat([df[:4], df[10:15]], axis=1)    # 把index相同的拼接成一行

In [ ]:
df6 = pd.concat([df[:4], df[10:15].reset_index(drop=True)], axis=1)

In [ ]:
df6

In [ ]:
df6.columns

In [ ]:
df6['cate_id']

### 缺失值（Missing Value）处理

In [ ]:
df.shape

In [ ]:
df.dropna().shape    # 丢弃NaN值

In [ ]:
df.fillna(0).head(10)    # 将NaN填充成固定值

In [ ]:
df.fillna(df['brand'].mean()).head(10)    # 使用brand均值填充

In [ ]:
df.fillna(method='ffill').head(10)    # 将NaN填充成前一个值

### 异常值 / 离群值（Outlier Data）处理

In [ ]:
price_des = df['price'].describe()
price_des

In [ ]:
valid_max = price_des['50%'] + 3 * (price_des['75%'] - price_des['50%'])
valid_min = price_des['50%'] - 3 * (price_des['50%'] - price_des['25%'])
df['price'] = df['price'].clip(valid_min, valid_max)    # 把异常值变为边界值
df['price'].describe()